# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/arabic_dialogue_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq

In [3]:
 data_set = load_dataset("CUTD/arabic_dialogue_df")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


arabic_dialogue_df.csv:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [4]:
data_set = data_set['train'].train_test_split(test_size=0.2)

In [5]:
data_set = data_set.map(remove_columns=["Unnamed: 0"])

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [6]:
data_set

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['dialogue', 'summary'],
        num_rows: 3000
    })
})

In [7]:
data_set['train'][3]

{'dialogue': 'إلين: ماذا حصلت على عيد الميلاد؟\n دونا: المعتاد ... زوجان من الكتب\n دونا: سترة\n دونا: العطور\n إلين: لطيف\n دونا: أنت؟\n إلين: إلى حد كبير نفس الشيء و .......\n دونا: ؟؟؟\n إلين: تذاكر لروما !!\n دونا: واو !!!',
 'summary': 'لعيد الميلاد ، حصلت دونا على اثنين من الكتب ، سترة وعطور.حصلت إيلين على بعض الأشياء والتذاكر المماثلة لروما.'}

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [8]:
checkpoint ='UBC-NLP/AraT5-base'
tokenizer =  AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [9]:
def pre (data):

  inputs=['summarize:'+x for x in data['dialogue']]
  inputs_tok=tokenizer(inputs,max_length = 256,padding='max_length',truncation=True)

  with tokenizer.as_target_tokenizer():
      leb = tokenizer(data["summary"], max_length=128, truncation=True, padding='max_length')

  inputs_tok["labels"] = leb["input_ids"]
  return inputs_tok


## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [10]:
# تاخذ البيانات ك batch random data
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [11]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [13]:
from transformers import Trainer,TrainingArguments

In [35]:
arg = Seq2SeqTrainingArguments(
    output_dir='output',
    learning_rate=0.003,
    per_device_train_batch_size=8,
    num_train_epochs = 3,
)

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [15]:
train_data = data_set.map(pre, batched=True, remove_columns=["dialogue", "summary"])

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [16]:
train_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [36]:
training = Seq2SeqTrainer(
    model=model,
    args=arg,
    data_collator =data_collator,
    train_dataset=train_data['train'],
    eval_dataset= train_data['test']
    )

## Step 8: Fine-tune the Model

Train the model using the specified arguments and dataset.

In [90]:
training.train()

Step,Training Loss


KeyboardInterrupt: 

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [30]:
text="في يوم الاحد, الموافق الخامس عشر من سيبتمبر, اختبرنا اختبارا عمليا وتم تتقسيمن لمجموعات وبحمد الله استطعنا ان نأخذ الدرجة الكاملة"

In [31]:
inputs = tokenizer('summarize:'+text, return_tensors="pt").input_ids

In [32]:
inputs

tensor([[ 35880,   4633,  16410,     33,    552,    107,   2614,     10,   6737,
           3415,    963,      6, 100413,  53035,     10,  51027,     54,  65970,
          21375,   1058,      3,     38,  87408,     54,  78867,     62,  41260,
             18,  60635,     37,  39527,   4684,   9369,      1]])

In [33]:
sumir = model.generate(inputs)

In [39]:
print(tokenizer.decode(sumir[0], skip_special_tokens=True))

🚫 Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay Malay
